# Fossil Data Extraction Baselines

This notebook sets up, runs and evaluates the baseline models for the fossil data extraction task.

The data and baseline approaches are as follows:

| **Entity Name**            | **Baseline Approach**                                              |
|:---:|:---|
| Geographic Location - GEOG | Regular Expressions (Goring et. al 2021)                                      |
| Site Name - SITE           | spaCy Pre-Trained NER model identifying location entities |
| Taxa - TAXA                | In-text search for existing taxa already in Neotoma                |
| Age - AGE                  | Regular Expressions (Goring et. al 2021)                                      |
| Altitude - ALTI            | Regular Expressions ("above sea level", "a.s.l.")                  |
| Email Address(es) - EMAIL  | Regular Expressions                                                |


In [11]:
import os, sys

import re
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

# ensure that the parent directory is on the path for relative imports
sys.path.append(os.path.join(os.path.abspath(''), ".."))

from src.entity_extraction.baseline_entity_extraction import (
    extract_geographic_coordinates,
    extract_site_names,
    extract_taxa,
    extract_age,
    extract_altitude,
    extract_email,
    baseline_extract_all
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Geographic Location - GEOG

The coordinates of a site are often reported in the literature in a variety of formats. Few varying examples are:
- 402646302N 0795855903W
- 40:26:46.302N 079:58:55.903W
- 40°26′46″N 079°58′56″W
- 40d 26′ 46″ N 079d 58′ 56″ W
- 40.446195N 79.982195W
- 40.446195, -79.982195
- 40.446195,-79.982195
- 40° 26.7717, -79° 58.93172
- N40:26:46.302 W079:58:55.903
- N40°26′46″ W079°58′56″
- N40d 26′ 46″ W079d 58′ 56″
- N40.446195 W79.982195
- 52°05.75′N, 131°13.25′W
- 10°50'E, 46°51'N

The baseline solution based off of Goring et. al 2021 uses regular expressions to identify coordinates. We've adpated and updated those regex patterns to reduce the number of false positive generated:
1. Pattern for geographic coordinates - ```[-+]?[NESW\d]+\s?[NESWd\.:°o◦'`"″]\s?[NESW]?\d{1,7}\s?[NESWd\.:°o◦′'`"″]?\s?\d{1,6}[[NESWd\.:°o◦′'`"″]?\s?[NESW]?```

## Site Name - SITE

We can use pretrained ner models to extract location names from literature, but we are not sure whether these entities will correspond to `region_name` or `site_name`

## Taxa - TAXA

The baseline approach to extract taxas from full text journal articles is to perform string matching. Using an exhaustive list of all the taxas is present on the neotoma database, we search for the exact taxon names in literature. 

## Age - AGE

The age of samples is often reported in the literature in a variety of formats.  The most common formats are:
- years BP - before present
- kyr BP - 1000’s of years BP
- ka BP - kilo annum BP
- a BP - annum BP
- Ma BP - million years BP
- YBP - years BP

In Neotoma there are three age columns, we have ageold, agetype and ageyoung.

- agetype: Age type or units. Includes the following:
  - Calendar years AD/BC
  - Calendar years BP
  - Calibrated radiocarbon years BP
  - Radiocarbon years BP
  - Varve years BP

The baseline solution based off of Goring et. al 2021 uses regular expressions to:
1. Identify the age entity in the sentence - `" BP "`
2. Determine if it is a range of dates - `"(\\d+(?:[.]\\d+)*) ((?:- {1,2})|(?:to)) (\\d+(?:[.]\\d+)*) ([a-zA-Z]+,BP"`
3. Extract the age entity from the sentence - `"(\\d+(?:[.]\\d+)*),((?:- {1,2})|(?:to)),(\\d+(?:[.]\\d+)*),([a-zA-Z]+,BP),"`

In [ ]:
test_sentences = [
    "1234 BP",
    "1234 Ma BP",
    "1234 to 1235 BP",
    "1234 - 1235 BP",
    "1234 -- 1235 BP",
    "1234 BP and 456 to 789 BP",
    "1234 BP and 456 to 789 Ma BP",
]

expected_results = [
    [{'start': 0, 'end': 7, 'label': ['AGE'], 'text': '1234 BP'}],
    [{'start': 0, 'end': 10, 'label': ['AGE'], 'text': '1234 Ma BP'}],
    [{'start': 0, 'end': 15, 'label': ['AGE'], 'text': '1234 to 1235 BP'}],
    [{'start': 0, 'end': 14, 'label': ['AGE'], 'text': '1234 - 1235 BP'}],
    [{'start': 0, 'end': 15, 'label': ['AGE'], 'text': '1234 -- 1235 BP'}],
    [
        {'start': 0, 'end': 7, 'label': ['AGE'], 'text': '1234 BP'}, 
        {'start': 12, 'end': 25, 'label': ['AGE'], 'text': '456 to 789 BP'}
    ],
    [
        {'start': 0, 'end': 7, 'label': ['AGE'], 'text': '1234 BP'}, 
        {'start': 12, 'end': 28, 'label': ['AGE'], 'text': '456 to 789 Ma BP'}
    ],
]

In [ ]:
# test that all the test sentences are extracted correctly
for i, sentence in enumerate(test_sentences):

    extracted_ages = extract_age(sentence)

    print(f"Testing sentence: {sentence}")
    print(f"Got: {extracted_ages}\n")
    assert extracted_ages == expected_results[i]

## Altitude - ALTI

To identify altitude descriptions the primary indicators are:
- "above sea level"
- "a.s.l."
- a single m as the last character after numbers or as a standalone word

In [ ]:
test_sentences = [
    "120m above sea level",
    "120m a.s.l.",
    "120 m above sea level",
    "120 m a.s.l.",
    "120m asl",
    "120 m asl",
    "The site was 120m above sea level",
    "The site was 120m a.s.l.",
    "The site was 120 m above sea level",
    "The site was 120 m a.s.l.",
    "First site was 120m asl and the second was 300 m asl",
]

expected_results = [
    [{'start': 0, 'end': 20, 'label': ['ALTI'], 'text': '120m above sea level'}],
    [{'start': 0, 'end': 11, 'label': ['ALTI'], 'text': '120m a.s.l.'}],
    [{'start': 0, 'end': 21, 'label': ['ALTI'], 'text': '120 m above sea level'}],
    [{'start': 0, 'end': 12, 'label': ['ALTI'], 'text': '120 m a.s.l.'}],
    [{'start': 0, 'end': 8, 'label': ['ALTI'], 'text': '120m asl'}],
    [{'start': 0, 'end': 9, 'label': ['ALTI'], 'text': '120 m asl'}],
    [{'start': 13, 'end': 33, 'label': ['ALTI'], 'text': '120m above sea level'}],
    [{'start': 13, 'end': 24, 'label': ['ALTI'], 'text': '120m a.s.l.'}],
    [{'start': 13, 'end': 34, 'label': ['ALTI'], 'text': '120 m above sea level'}],
    [{'start': 13, 'end': 25, 'label': ['ALTI'], 'text': '120 m a.s.l.'}],
    [
        {'start': 15, 'end': 23, 'label': ['ALTI'], 'text': '120m asl'},
        {'start': 43, 'end': 52, 'label': ['ALTI'], 'text': '300 m asl'}
    ]
]

In [ ]:
# test that all the test sentences are extracted correctly
for i, sentence in enumerate(test_sentences):

    extracted_altitude = extract_altitude(sentence)

    print(f"Testing sentence: {sentence}")
    print(f"Found: {extracted_altitude}\n")
    assert extracted_altitude == expected_results[i]

## Email Addresses - EMAIL

There are existing regex patterns developed to identify emails. The one used below was sourced from this StackoverFlow thread: 
- https://stackoverflow.com/questions/201323/how-can-i-validate-an-email-address-using-a-regular-expression

In [ ]:
test_sentences = [
    "ty.elgin.andrews@gmail.com",
    "john.smith@aol.com",
    "ty.andrews@student.ubc.ca",
    # from GGD 54b4324ae138239d8684a37b segment 0
    "E-mail addresses : carina.hoorn@milne.cc (C. Hoorn -) mauro.cremaschi@libero.it"
]

expected_results = [
    [{'start': 0, 'end': 26, 'label': ['EMAIL'], 'text': 'ty.elgin.andrews@gmail.com'}],
    [{'start': 0, 'end': 18, 'label': ['EMAIL'], 'text': 'john.smith@aol.com'}],
    [{'start': 0, 'end': 25, 'label': ['EMAIL'], 'text': 'ty.andrews@student.ubc.ca'}],
    [
        {'start': 19, 'end': 40, 'label': ['EMAIL'], 'text': 'carina.hoorn@milne.cc'},
        {'start': 54, 'end': 79, 'label': ['EMAIL'], 'text': 'mauro.cremaschi@libero.it'}
    ]
]

In [ ]:
for i, sentence in enumerate(test_sentences):

    extracted_emails = extract_email(sentence)

    print(f"Testing sentence: {sentence}")
    print(f"Found: {extracted_emails}\n")
    assert extracted_emails == expected_results[i]

# Evaluation of Baseline Methods

Multiple methods to evaluate the NER task are evaluated for a given report:
1. Does the number instances of each category align with what is present in Neotoma?
   1. For each article need to pull out the number of instances of each category for the given DOI/GDD id
2. Does the number of instances of each category align with what has been labelled?
   1. Using the team labelled reports, count the number of instances of each category for each report.
3. Does the extracted locations align with the labelled locations?
   1. This can be done with the Python package `seqeval` which is used to evaluate the performance of a sequence labeling tasks. ([`seqeval` Github](https://github.com/chakki-works/seqeval))

## Loading Raw & Labelled Data

In [ ]:
from collections import defaultdict

labelled_file_path = os.path.join(os.getcwd(), os.pardir, "data", "labelled", "labelling-labelled")

## Comparison to Neotoma Data



## Comparison of Entity Counts to Labelled Data

In [ ]:
labelled_files = os.listdir(labelled_file_path)

# count the number of occurences of each label
annotated_label_counts = defaultdict(int)
baseline_label_counts = defaultdict(int)

for file in labelled_files:
    
    with open(os.path.join(labelled_file_path, file), "r") as f:
        task = json.load(f)

    raw_text = task['task']['data']['text']
    
    # get the baseline annotations
    baseline_result = baseline_extract_all(raw_text)
    for baseline in baseline_result:
        baseline_label_counts[baseline['label'][0]] += 1

    annotation_result = task['result']
    
    for annotation in annotation_result:
        annotated_label_counts[annotation['value']['labels'][0]] += 1

In [ ]:
# plot the percentage of entities extracted by baseline vs annotated
annotated_labels = list(annotated_label_counts.keys())
annotated_counts = list(annotated_label_counts.values())

baseline_counts = [baseline_label_counts[label] for label in annotated_labels]

annotated_counts = np.array(annotated_counts)
baseline_counts = np.array(baseline_counts)

# make into a tidy dataframe with columns Label, Source, Count
annotated_df = pd.DataFrame(
    {
        'Label': annotated_labels + annotated_labels,
        'Source': ['Annotated'] * len(annotated_labels) + ['Baseline'] * len(annotated_labels),
        'Count': np.concatenate([annotated_counts, baseline_counts])
    }
)

fig = px.bar(
    annotated_df,
    x="Label",
    y="Count",
    color="Source",
    barmode='group',
    # labels={'x': 'Labels', 'value': 'No. of Entities'},
    title='Counts of Labels in Annotated and Baseline Results',
    width=800,
).update_layout(
    xaxis={'categoryorder': 'total descending'}, 
    margin={'l': 0, 'r': 0, 't': 50, 'b': 0}, 
)

fig.show()

In [ ]:
percentage_extracted = baseline_counts / annotated_counts

fig = px.bar(
    x=annotated_labels, 
    y=percentage_extracted, 
    color=annotated_labels,
    labels={'x': 'Labels', 'y': 'Percentage Extracted'},
    title='Percentage of Entities Extracted by Baseline vs Annotated',
    width=800,
    text=np.round(percentage_extracted, 3)*100,
    hover_name=annotated_labels,
).update_layout(
    xaxis={'categoryorder': 'total descending'},
    margin={'l': 0, 'r': 0, 't': 50, 'b': 0},
    yaxis={'tickformat': ',.0%'},
    showlegend=False,
    yaxis_range=[0, 1],
)
fig.show()
